# EXTRACCION, TRANSFORMACIÓN Y LIMPIEZA 

## Base de datos: Movies

Se utiliza la libreria de pandas para dar inicio a la estracción y tratamiento de los datos.

In [2]:
import pandas as pd
import numpy as np
import json
import ast
import os

Inicio la extracción, primero del detaset "movies" y luego del dataset "credits" para despues unirlos. 

In [91]:
# Extraigo el dataset "movies" y reviso el tamaño
df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
df_movies.shape

C:\Users\anavi\AppData\Local\Temp\ipykernel_16820\131553651.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')


(45466, 24)

In [92]:
# Extraigo los dataset "credits" y reviso el tamaño
df_credits = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')
df_credits.shape

(45476, 3)

In [93]:
# Quiero saber el tamaño de mi dataset para reducirlo hasta llegar a 100 megas
tamaño1 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
tamaño2 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')

print(f"Tamaño movies: {tamaño1 / (1024 * 1024):.2f} MB y tamaño credits: {tamaño2 / (1024 * 1024):.2f} MB")

Tamaño movies: 32.85 MB y tamaño credits: 181.12 MB


Como veo que el dataset credits es mucho más grande que el dataset movies entonces empiezo limpiandolo para reducirlo al máximo.

In [94]:
df_credits[df_credits.duplicated(keep=False)].shape # Verifico si hay datos duplicados para eliminarlos posteriormente

(73, 3)

In [95]:
tamaño_en_bytes = df_credits.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 256.51 MB.


In [96]:
df_credits = df_credits.drop_duplicates() # Elimino los datos duplicados en mi dataset original
df_credits[df_credits.duplicated(keep=False)].shape #Verifico que hayan quedado eliminados

(0, 3)

In [97]:
df_credits.columns # Verifico que columnas existen en mi data set

Index(['cast', 'crew', 'id'], dtype='object')

In [98]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Mi dataset tiene dos columas. La primera contiene el elenco "cast" y la segunda contiene los nombres del equipo "crew". Voy a empezar por depurar la columna "cast" y dejar unicamente 5 actores principales, respetando el orden en el que vienen.

In [99]:
type(df_credits['cast'][0]) # se verifica el tipo de datos, deben ser listas o dicccionarios para poder normalizarlos

str

In [100]:
#Se convierte el tipo de datos de String a Diccionario o lista

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['cast'] = df_credits['cast'].apply(convertir_a_diccionario)

type(df_credits['cast'][0])

list

In [101]:
# Se muestran los tipos de datos únicos en la columnas

df_credits['cast'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [122]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['cast']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_cast = pd.concat(expanded_dfs, ignore_index=True)

In [123]:
df_cast.columns #Para verificar las columnas existentes en el nuevo data frame

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order',
       'profile_path'],
      dtype='object')

In [124]:
df_cast.shape #Para verificar el tamaño del nuevo data frame

(562132, 8)

In [125]:
df_cast = df_cast[df_cast['order'].between(1, 5)] # Filtrar los primeros 5 actores y eliminar el resto

In [126]:
df_cast.tail(10)

,cast_id,character,credit_id,gender,id,name,order,profile_path
562112,6.0,Emily Shaw,52fe4776c3a368484e0c83a3,1.0,67758,Erika Eleniak,1,/7ZdUGY78qpB2MkTzAQvYEYzncwN.jpg
562113,7.0,Det. Mark Winston,52fe4776c3a368484e0c83a7,2.0,67758,Adam Baldwin,2,/vhZ8AD36h09mBLuyIXboglRLgIu.jpg
562114,8.0,Jayne Ferré,52fe4776c3a368484e0c83ab,1.0,67758,Julie du Page,3,/gBLzn3Z2S7Hx7A0stZBi7mk9fpD.jpg
562115,10.0,Alex Tyler,52fe4776c3a368484e0c83b3,2.0,67758,James Remar,4,/AeSuQlBclvWqztlpkf9nIhvvhsc.jpg
562116,11.0,Tony,52fe4776c3a368484e0c83b7,2.0,67758,Damian Chapa,5,/e00LyDBIhDOFWgwfSU0wCTLL4ax.jpg
562127,2.0,,52fe4ea59251416c7515d7d5,2.0,227506,Iwan Mosschuchin,1,None
562128,3.0,,52fe4ea59251416c7515d7d9,1.0,227506,Nathalie Lissenko,2,None
562129,4.0,,52fe4ea59251416c7515d7dd,2.0,227506,Pavel Pavlov,3,None
562130,5.0,,52fe4ea59251416c7515d7e1,0.0,227506,Aleksandr Chabrov,4,None
562131,6.0,,52fe4ea59251416c7515d7e5,1.0,227506,Vera Orlova,5,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [127]:
df_cast = df_cast[['id'] + [col for col in df_cast.columns if col != 'id']] #Dejo de primera la columna "id"
df_cast = df_cast.drop(columns=['credit_id', 'gender', 'cast_id', 'profile_path']) #Elimino las columnas innecesarias

In [133]:
df_cast['order'] = df_cast['order'].apply(lambda x: f"Character {x}")
df_cast.rename(columns={'id': 'Id', 'name': 'Name', 'character': 'Name Character', 'order': 'Rol'}, inplace=True)


In [134]:
df_cast.shape #Para verificar el tamaño nuevamente

(202338, 4)

In [136]:
df_cast.head(10)

,Id,Name Character,Name,Rol
0,862,Woody (voice),Tom Hanks,Character 1
1,862,Buzz Lightyear (voice),Tim Allen,Character 2
2,862,Mr. Potato Head (voice),Don Rickles,Character 3
3,862,Slinky Dog (voice),Jim Varney,Character 4
4,862,Rex (voice),Wallace Shawn,Character 5
13,8844,Alan Parrish,Robin Williams,Character 1
14,8844,Samuel Alan Parrish / Van Pelt,Jonathan Hyde,Character 2
15,8844,Judy Sheperd,Kirsten Dunst,Character 3
16,8844,Peter Shepherd,Bradley Pierce,Character 4
17,8844,Sarah Whittle,Bonnie Hunt,Character 5


In [138]:
df_cast.shape

(202338, 4)

In [139]:
# Calcular el uso de memoria del DataFrame en bytes
uso_memoria = df_cast.memory_usage(deep=True).sum()

# Convertir el uso de memoria a megabytes
uso_memoria_mb = uso_memoria / (1024 * 1024)

print(f"El DataFrame utiliza aproximadamente {uso_memoria_mb:.2f} MB de memoria.")


El DataFrame utiliza aproximadamente 43.36 MB de memoria.


In [140]:
tamaño_en_bytes = df_cast.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 43.36 MB.


Una vez eliminadas las celdas que no se van a usar de la columna cast y creado un data set con este nombre, se procede a trabajar con la columna "Crew" de la cual solo se va a conservar el Director. 

In [142]:
df_credits['crew'].head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [143]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['crew'] = df_credits ['crew'].apply(convertir_a_diccionario)

type(df_credits['crew'][0])

list

In [144]:
# Se muestran los tipos de datos únicos en la columna

df_credits['crew'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [145]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['crew']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_crew = pd.concat(expanded_dfs, ignore_index=True)


In [146]:
df_crew.head() #Para ver como quedó estructurado el dataset creado. 

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [147]:
df_crew.shape #Para confirmar el tamaño (filas y columnas)

(463927, 7)

In [148]:
# Para revisar el tamaño en megas. 

tamaño_en_bytes = df_crew.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 150.73 MB.


In [149]:
df_crew = df_crew[df_crew['job'] == 'Director']

In [150]:
df_crew.columns

Index(['credit_id', 'department', 'gender', 'id', 'job', 'name',
       'profile_path'],
      dtype='object')

In [155]:
#df_crew = df_crew.drop(columns=['credit_id', 'profile_path', 'department', 'gender'])
df_crew.rename(columns={'id': 'Id', 'job': 'Rol', 'name': 'Name'}, inplace=True)

In [156]:
df_crew.head()

,Id,Rol,Name
0,862,Director,John Lasseter
109,8844,Director,Joe Johnston
122,15602,Director,Howard Deutch
126,31357,Director,Forest Whitaker
141,11862,Director,Charles Shyer


In [162]:
df_crew.shape

(49008, 3)

Ya tengo ambos dataset depurados, ahora voy a combinarlos para llegar a uno solo

In [158]:
df_combined = pd.concat([df_cast, df_crew], axis=0, ignore_index=True)

In [160]:
df_combined[df_combined['Id'] == 862]

,Id,Name Character,Name,Rol
0,862,Woody (voice),Tom Hanks,Character 1
1,862,Buzz Lightyear (voice),Tim Allen,Character 2
2,862,Mr. Potato Head (voice),Don Rickles,Character 3
3,862,Slinky Dog (voice),Jim Varney,Character 4
4,862,Rex (voice),Wallace Shawn,Character 5
202338,862,NaN,John Lasseter,Director


In [163]:
df_combined.shape

(251346, 4)

In [161]:
tamaño_en_bytes = df_combined.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 50.10 MB.
